In [1]:
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import requests
from datetime import datetime

load_dotenv()
dbUrl = os.environ["ODOO_URL"]
dbName = os.environ["ODOO_NAME"]
dbUsername = os.environ["ODOO_USERNAME"]
dbPassword = os.environ["ODOO_PASSWORD"]
erAppId = os.environ["APP_ID"]

DATABASE_CONNECTION_URI = os.environ["DB_URL"]
# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

class OdooAPI:
    def __init__(self):
        self.dbUrl = dbUrl
        self.dbName = dbName
        self.dbUsername = dbUsername
        self.dbPassword = dbPassword
        self.common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(self.dbUrl))
        self.uid = self.common.authenticate(self.dbName, self.dbUsername, self.dbPassword, {})
        self.models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(self.dbUrl))
        self.country_codes = {
            "Argentina": "AR",
            "Paraguay": "PY",
            "Bolivia": "BO",
            "Uruguay": "UY",
            "Chile": "CL",
            "Perú": "PE",
            "Peru": "PE",
            "Ecuador": "EC",
            "Colombia": "CO",
            "Venezuela": "VE",
            "Brasil": "BR",
            "Panamá": "PA",
            "Panama": "PA",
            "Costa Rica": "CR",
            "Honduras": "HN",
            "El Salvador": "SV",
            "Guatemala": "GT",
            "México": "MX",
            "Mexico": "MX",
            "República Dominicana": "DM",
            "Republica Dominicana": "DM",
            "Rep Dom": "DM",
            "Puerto Rico": "PR",
            "España": "ES",
        }

    # Data Handling Functions
    def id_field_into_1value(self, jsonFile):
        for index, line in enumerate(jsonFile):
            for item in line:
                if type(line[item]) == list:
                    jsonFile[index][item] = jsonFile[index][item][1]

    # Export to CSV functions
    def raw_export_subscriptions(self, properties_dict):
        subscriptions = self.models.execute_kw(self.dbName,
                                               self.uid,
                                               self.dbPassword,
                                               'sale.order',
                                               'search_read',
                                               [[["stage_id", "=", 9]]],
                                               {'fields': list(properties_dict.keys())})
        
        self.id_field_into_1value(subscriptions)
        df = pd.DataFrame(subscriptions)
        # insert the dataframe into the postgres table
        df.to_sql('test', con=engine, if_exists='replace', index=False)
        #df.to_csv("raw_odoo_subscriptions.csv", encoding="latin-1", index=False, errors='ignore')

    def handle_raw_odoo(self, csv_file, properties_dict={}, values_dict={}, date_columns=[]):
        # Csv into Dataframe
        df = pd.read_csv(csv_file + ".csv", encoding="latin-1")
        # Rename Columns
        df.rename(columns=properties_dict, inplace=True)
        # Rename Values
        df = df.applymap(lambda x: str(x) if pd.notnull(x) else '')
        df.replace(values_dict, inplace=True)
        # Columns into lower case and underscore union
        columns = [column.lower() for column in df.columns]
        columns = ["_".join(column.split(" ")) for column in columns]
        df.columns = columns
        # Date Standarization
        date_columns = [char.lower() for char in date_columns]
        date_columns = ["_".join(char.split(" ")) for char in date_columns]
        for date_column in date_columns:
            df[date_column] = pd.to_datetime(df[df[date_column].notna()][date_column], errors='raise')
            df[date_column] = df[date_column].dt.strftime('%Y-%m-%d')
        df.to_csv(csv_file[4:] + ".csv", encoding="latin-1", index=False, errors='ignore')

In [2]:
o = OdooAPI()

properties_dict = {
  "create_date": 1,
  "name": 1,
  #"order_line": 1
}

o.raw_export_subscriptions(properties_dict)

